In [82]:
import urllib.request as urllib2
import re
from bs4 import BeautifulSoup
import pandas as pd

In [203]:
fighter_name = input("Please input fighter's name and code separated by '-':")

Please input fighter's name and code separated by '-':Conor-McGregor-29688


In [204]:
url = "http://www.sherdog.com/fighter/" + fighter_name
print(url)

http://www.sherdog.com/fighter/Conor-McGregor-29688


In [205]:
#redirect to the web page for desired fighter
webpage = urllib2.urlopen(url)
soup = BeautifulSoup(webpage)

C:\Users\Arko\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Arko\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [130]:
print(soup.prettify())

<!DOCTYPE html>
<html version="XHTML+RDFa 1.0">
 <head>
  <title>
   Michael "The Count" Bisping MMA Stats, Pictures, News, Videos, Biography - Sherdog.com
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="en-US" http-equiv="Content-Language"/>
  <meta content="Sherdog.com" name="author"/>
  <meta content="Sherdog.com" name="publisher"/>
  <meta content="2017 - Sherdog.com" name="copyright"/>
  <meta content="index, follow" name="robots"/>
  <meta content="Michael Bisping official Sherdog mixed martial arts stats, photos, videos, breaking news, and more for the Middleweight fighter from England." name="description"/>
  <meta content="!" name="fragment"/>
  <meta content="100000399936875" property="fb:admins"/>
  <meta content="Michael Bisping official Sherdog mixed martial arts stats, photos, videos, breaking news, and more for the Middleweight fighter from England." property="og:description"/>
  <meta biography

Now that we have the html content for the fighter we can find the desired elements like Name, Age, etc.
Sometimes, we need to find an inner html within the outer depending upon the tag

In [206]:
fighterName = soup.find('span', class_ = 'fn').get_text()
nickname = soup.find('span', class_ = 'nickname').get_text()
birthdate = soup.find('span',itemprop = 'birthDate').get_text()
age = soup.find('span',class_ = 'item birthday').find('strong').get_text()
locality = soup.find(itemprop = "addressLocality").get_text()
nationality = soup.find(itemprop = "nationality").get_text()
association = soup.find(class_ = 'item association').find('span', itemprop = 'name').get_text()
height = soup.find(class_ = 'item height').find('strong').get_text()
weight = soup.find(class_ = 'item weight').find('strong').get_text()
weightClass = soup.find(class_ = 'item wclass').find('strong').get_text()
wins = soup.find('span',class_='counter').get_text()

For getting the table for Fight History-Pro we need to first distinguish between Amateur fights and Pro Fights. Hence, first we distinguish between these two using the String tags: Amateur Fights and Fight History. 
Following this, we then move up the html tag for which we use findParent and finally we access the table tag to get contents of the table

In [207]:
pro_fight_history_table = soup.find(string = 'Fight History - Pro').findParent(class_ = 'module fight_history').find('table')

In [198]:
pro_fight_history_table

<table border="1">
<tr class="table_head">
<td class="col_one">Result</td>
<td class="col_two">Fighter</td>
<td class="col_three">Event</td>
<td class="col_four">Method/Referee</td>
<td class="col_five">R</td>
<td class="col_six">Time</td>
</tr>
<tr class="odd">
<td><span class="final_result win">win</span></td>
<td><a href="/fighter/Dominick-Cruz-12107">Dominick Cruz</a></td>
<td><a href="/events/UFC-207-Nunes-vs-Rousey-55005"><span itemprop="award">UFC 207 - Nunes vs. Rousey</span></a><br/><span class="sub_line">Dec / 30 / 2016</span></td>
<td>Decision (Unanimous)<br/><span class="sub_line">John McCarthy</span></td>
<td>5</td>
<td>5:00</td>
</tr>
<tr class="even">
<td><span class="final_result win">win</span></td>
<td><a href="/fighter/Takeya-Mizugaki-12074">Takeya Mizugaki</a></td>
<td><a href="/events/UFC-202-Diaz-vs-McGregor-2-52567"><span itemprop="award">UFC 202 - Diaz vs. McGregor 2</span></a><br/><span class="sub_line">Aug / 20 / 2016</span></td>
<td>TKO (Punches)<br/><span cl

Now we can extract the individual elements from the table element.The rows of the table are identified by the css classes odd and even

In [208]:
rows = pro_fight_history_table.findAll(class_={'odd','even'})
#We will store each of the attributes in separate lists
result = [];opponent = [];event_url=[];event_name=[];event_date=[];referee=[];method=[];rounds=[];timeofmatch=[];
for row in rows:
    cells = row.findAll('td')
    result.append(cells[0].get_text())
    opponent.append(cells[1].get_text())
    event_url.append(cells[2].find('a').get('href'))
    event_name.append(cells[2].find('a').get_text())
    event_date.append(cells[2].find(class_='sub_line').get_text())
    referee.append(cells[3].find('span',class_='sub_line').get_text())
    rounds.append(cells[4].get_text())
    timeofmatch.append(cells[5].get_text())

In [202]:
print(referee)

['John McCarthy', 'John McCarthy', 'John McCarthy', 'Mario Yamasaki', 'Chris Tognoni', 'Herb Dean', 'N/A', 'Bill Bookwalter', 'N/A', 'James Chappell', 'Chip Snider']


In [209]:
#merge the above attributes to one dataframe
final_data = pd.DataFrame()
final_data = pd.DataFrame({'result':result,
                        'opponent':opponent,
                         'event_url':event_url,
                          'event_name':event_name,
                           'event_date':even_date,
                            'referee':referee,
                             'rounds':rounds,
                              'timeofmatch':timeofmatch})
                         

In [210]:
final_data.head()

,event_date,event_name,event_url,opponent,referee,result,rounds,timeofmatch
0,Mar / 08 / 2008,UFC 205 - Alvarez vs. McGregor,/events/UFC-205-Alvarez-vs-McGregor-52249,Eddie Alvarez,John McCarthy,win,2,3:04
1,Mar / 08 / 2008,UFC 202 - Diaz vs. McGregor 2,/events/UFC-202-Diaz-vs-McGregor-2-52567,Nate Diaz,John McCarthy,win,5,5:00
2,Mar / 08 / 2008,UFC 196 - McGregor vs. Diaz,/events/UFC-196-McGregor-vs-Diaz-47931,Nate Diaz,Herb Dean,loss,2,4:12
3,Mar / 08 / 2008,UFC 194 - Aldo vs. McGregor,/events/UFC-194-Aldo-vs-McGregor-42245,Jose Aldo,John McCarthy,win,1,0:13
4,Mar / 08 / 2008,UFC 189 - Mendes vs. McGregor,/events/UFC-189-Mendes-vs-McGregor-42211,Chad Mendes,Herb Dean,win,2,4:57
